# Consultas oficiales SAC

Adquirente por día por clave de transacción

Archivo Excel Hoja Adq\_ventas\_devs

In [ ]:
--Adquirente por dia por cve de transaccion

use sac_mar

select 
a.fecha,
b.nombre_banco_grupo,
a.[cve_txn],
sum(a.[cantidad]) as txns,
sum(a.[IMPORTE]) as total
into #temp_totales
from INFO_SAC as a
inner join sac.dbo.adquirentes as b on a.id_adquirente =b.id_adquirente
--where fecha>='20221230'
where a.id_adquirente=136
group by a.fecha,
b.nombre_banco_grupo,
a.[cve_txn]

select a.fecha,
a.nombre_banco_grupo,
(select sum(txns) from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=101) as ventas_credito_txns,
(select sum(total) from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=101) as ventas_credito_total,
(select isnull(sum(txns),0)  from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=108) as devs_credito_txns,
(select isnull(sum(total),0) from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=108) as devs_credito_total,
(select sum(txns) from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=103) as ventas_debito_txns,
(select sum(total) from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=103) as ventas_debito_total,
(select isnull(sum(txns),0)  from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=109) as devs_debito_txns,
(select isnull(sum(total),0) from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=109) as devs_debito_total,
(select isnull(sum(txns),0) from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=106) as cashback_txns,
(select isnull(sum(total),0) from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=106) as cashback_total,
sum(txns) as txns,
sum(total) as total
from #temp_totales as a
group by a.fecha,a.nombre_banco_grupo

drop table #temp_totales

## Suma de totales

Hoja de excel: Adq\_totales

In [ ]:
--totales

-- use sac_feb

--enero
select 
b.nombre_banco_grupo,
sum(a.[IMPORTE]) as total,
sum(a.[cantidad]) as txns
into #temp_totales_totales
from sac_ene.dbo.INFO_SAC as a
inner join sac.dbo.adquirentes as b on a.id_adquirente =b.id_adquirente
where a.id_adquirente=136
group by 
b.nombre_banco_grupo

--febrero
insert into #temp_totales_totales
select 
b.nombre_banco_grupo,
sum(a.[IMPORTE]) as total,
sum(a.[cantidad]) as txns
from sac_feb.dbo.INFO_SAC as a
inner join sac.dbo.adquirentes as b on a.id_adquirente =b.id_adquirente
where a.id_adquirente=136
group by 
b.nombre_banco_grupo


--marzo
insert into #temp_totales_totales
select 
b.nombre_banco_grupo,
sum(a.[IMPORTE]) as total,
sum(a.[cantidad]) as txns
from sac_mar.dbo.INFO_SAC as a
inner join sac.dbo.adquirentes as b on a.id_adquirente =b.id_adquirente
where a.id_adquirente=136
group by 
b.nombre_banco_grupo

select 
a.nombre_banco_grupo,
sum(total) as total,
sum(txns) as txns
from #temp_totales_totales as a
group by a.nombre_banco_grupo

drop table #temp_totales_totales

## Adquirente vs Emisor

Hoja de excel: Adq\_emisor

Resolver top 5000 registros en Azure DS, mientras tanto ejecutar en SSMS

In [ ]:
use BOS_DB

select a.fecha 
,aa.nombre_banco_grupo as id_adquirente
,a.cve_txn
,b.tipo_comercio
,a.cantidad
,a.importe
,case when C.bin6 is null then 
		case when left(a.prefijo,1)='2' then 'MC'
			when left(a.prefijo,1)='5' then 'MC'
			when left(a.prefijo,1)='4' then 'Visa'
		else 'Otros' end
	else c.INSTITUCION end as Emisor
into #temp
from sac_ene.dbo.info_sac as a
inner join sac.dbo.adquirentes as aa on a.id_adquirente =aa.id_adquirente
inner join ACT.[dbo].[ACT] as b on a.[NO_COMERCIO]=b.afiliacion       --se pierden los comercios que aun no están en ACT
left join sac.dbo.bines_emisor as C on left(prefijo,6)=C.bin6
--where fecha>='20221230'

select fecha,id_adquirente,cve_txn,tipo_comercio,Emisor,
sum(cantidad) as txns,
sum(importe) as total
from #temp
group by  fecha,id_adquirente,cve_txn,tipo_comercio,Emisor
order by fecha, id_adquirente


drop table #temp


## SAC vs ACT  Afiliaciones transaccionando

Hoja de excel: transaccionando

In [ ]:
-----SAC VS ACT Afiliaciones transaccionando

-- la tabla afil sac dependerá del periodo de tiempo consultado

Use sac_ene

select distinct [NO_COMERCIO] 
into #temp_sac
from sac_ene.[dbo].[INFO_SAC] 
--where fecha>='20221230'

--create table afil_sac(
--afiliacion varchar(10) null)

--delete from sac.dbo.afil_sac

insert into sac.dbo.afil_sac
select * from #temp_sac
where no_comercio not in (select afiliacion from sac.dbo.afil_sac)


select a.afiliacion,
	iif(b.afiliacion is null, 'No actualizado',b.Banco) as cve_banco,
	iif(b.afiliacion is null, 'No actualizado',b.tipo_comercio) as tipo_comercio,
	'SI' as transaccionando
		into #temp_sac1
from sac.dbo.afil_sac as A
left join ACT.[dbo].[ACT] as b on a.afiliacion=b.afiliacion


select a.cve_banco,tipo_comercio,transaccionando,count(*) as conteo, 
iif(b.cve_banco is null,'No actualizado',b.institucion) as institucion
into #SALIDA
		from #temp_sac1 as a
left join act.[dbo].[act_bancos] as b on a.cve_banco=b.cve_banco
group by a.cve_banco,tipo_comercio,transaccionando,b.institucion,b.cve_banco


select a.[Afiliacion],a.Banco,a.SIC,a.tipo_comercio,
		'No' as transaccionando
into #temp_act
from 
ACT.[dbo].[ACT] as a
where a.Afiliacion not in (
select afiliacion from sac.dbo.afil_sac)
and [Com Activo]='V'
and banco not in ('BCMR','BNMX')

insert into #SALIDA
select banco,tipo_comercio,transaccionando,count(*) as conteo,
b.institucion
from #temp_act as a
inner join act.[dbo].[act_bancos] as b on a.banco=b.cve_banco
group by a.banco,tipo_comercio,transaccionando,b.institucion--,b.cve_banco


select * from #SALIDA


drop table #temp_sac1
drop table #temp_act
drop table #SALIDA
drop table #temp_sac


### Adquirentes vs clientes

Hojas de excel: adq\_clientes, adq top 10 clientes importe, adq top 10 clientes txns, comercios vs prosa txns, comercios vs prosa importe

Resolver top 5000 registros en Azure DS, mientras tanto ejecutar en SSMS

In [ ]:
use SAC_ENE

--enero
select 
b.nombre_banco_grupo,
iif(c.Afiliacion is null,'NO ACTUALIZADA',c.clasif) as cliente,
sum(a.[cantidad]) as txns,
sum(a.[IMPORTE]) as total
into #temp_clientes
from sac_ene.dbo.INFO_SAC as a
inner join sac.dbo.adquirentes as b on a.id_adquirente =b.id_adquirente
left join ACT.dbo.[ACT] as c on a.no_comercio=c.afiliacion
group by b.nombre_banco_grupo,c.Afiliacion,c.clasif

select nombre_banco_grupo,
	cliente,
	sum(txns) as txns,
	sum(total) as total
	into #temp_clientes_agrupada
	from #temp_clientes
	group by nombre_banco_grupo,
	cliente
	order by txns desc

select * from #temp_clientes_agrupada
order by txns desc


--top 10 clientes por banco por totales
WITH TOPTEN AS (
    SELECT *, ROW_NUMBER() 
    over (
        PARTITION BY nombre_banco_grupo 
        order by total desc
    ) AS RowNo 
    FROM #temp_clientes_agrupada
)
SELECT * FROM TOPTEN WHERE RowNo <= 10

--top 10 clientes por banco por txns
WITH TOPTEN AS (
    SELECT *, ROW_NUMBER() 
    over (
        PARTITION BY nombre_banco_grupo 
        order by txns desc
    ) AS RowNo 
    FROM #temp_clientes_agrupada
)
SELECT * FROM TOPTEN WHERE RowNo <= 10

declare @total money
declare @txns as decimal(18,2)

select @txns=sum(txns),@total=sum(total) from #temp_clientes_agrupada

--clientes vs prosa txns
select ROW_NUMBER() OVER (ORDER BY sum(txns) desc) as top_txns, 
cliente, 
sum(txns) as txns,
sum(txns)/@txns as perc_txns
from #temp_clientes_agrupada
group by cliente

--clientes vs prosa importe
select ROW_NUMBER() OVER (ORDER BY sum(total) desc) as top_total, 
cliente, 
sum(total) as total,
sum(total)/@total as perc_total
from #temp_clientes_agrupada
group by cliente



drop table #temp_clientes
drop table #temp_clientes_agrupada

### Clientes por agregador

Hoja de excel: adq\_agregadores\_clientes

Resolver top 5000 registros en Azure DS, mientras tanto ejecutar en SSMS

In [ ]:
use sac_ene 
--clientes por agregador
--ENERO
select 
b.nombre_banco_grupo,
a.cve_txn,
iif(c.Afiliacion is null,'NO ACTUALIZADA',c.clasif) as agregador,
iif(c.Afiliacion is null,'NO ACTUALIZADA',a.comercio) as comercio,
sum(a.[cantidad]) as txns,
sum(a.[IMPORTE]) as total
into #temp_clientes_agreg
from sac.dbo.INFO_SAC as a
inner join sac.dbo.adquirentes as b on a.id_adquirente =b.id_adquirente
left join ACT.[dbo].[ACT] as c on a.no_comercio=c.afiliacion
where c.tipo_comercio='Agregador'
group by b.nombre_banco_grupo,a.cve_txn,c.Afiliacion,c.clasif,a.comercio

select a.nombre_banco_grupo,
a.agregador,
a.comercio,
sum(a.txns) as txns,
sum(a.total) as total,
(select isnull(sum(txns),0) from #temp_clientes_agreg where nombre_banco_grupo=a.nombre_banco_grupo and agregador=a.agregador and comercio=a.comercio and cve_txn=101) as ventas_credito_txns,
(select isnull(sum(total),0) from #temp_clientes_agreg where nombre_banco_grupo=a.nombre_banco_grupo and agregador=a.agregador and comercio=a.comercio and  cve_txn=101) as ventas_credito_total,
(select isnull(sum(txns),0) from #temp_clientes_agreg where nombre_banco_grupo=a.nombre_banco_grupo and agregador=a.agregador and comercio=a.comercio and  cve_txn=108) as devs_credito_txns,
(select isnull(sum(total),0) from #temp_clientes_agreg where nombre_banco_grupo=a.nombre_banco_grupo and agregador=a.agregador and comercio=a.comercio and  cve_txn=108) as devs_credito_total,
(select isnull(sum(txns),0) from #temp_clientes_agreg where nombre_banco_grupo=a.nombre_banco_grupo and agregador=a.agregador and comercio=a.comercio and  cve_txn=103) as ventas_debito_txns,
(select isnull(sum(total),0) from #temp_clientes_agreg where nombre_banco_grupo=a.nombre_banco_grupo and agregador=a.agregador and comercio=a.comercio and  cve_txn=103) as ventas_debito_total,
(select isnull(sum(txns),0) from #temp_clientes_agreg where nombre_banco_grupo=a.nombre_banco_grupo and agregador=a.agregador and comercio=a.comercio and  cve_txn=109) as devs_debito_txns,
(select isnull(sum(total),0) from #temp_clientes_agreg where nombre_banco_grupo=a.nombre_banco_grupo and agregador=a.agregador and comercio=a.comercio and  cve_txn=109) as devs_debito_total,
(select isnull(sum(txns),0) from #temp_clientes_agreg where nombre_banco_grupo=a.nombre_banco_grupo and agregador=a.agregador and comercio=a.comercio and  cve_txn=106) as cashback_txns,
(select isnull(sum(total),0) from #temp_clientes_agreg where nombre_banco_grupo=a.nombre_banco_grupo and agregador=a.agregador and comercio=a.comercio and cve_txn=106) as cashback_total
into #temp_clientes_agreg_agrupada
from #temp_clientes_agreg as a
group by a.nombre_banco_grupo,a.agregador,a.comercio
order by txns desc

select * from #temp_clientes_agreg_agrupada
order by total desc

drop table #temp_clientes_agreg
drop table #temp_clientes_agreg_agrupada

# Marcas

In [ ]:

--enero

select 
aa.nombre_banco_grupo,
case 
when b.bin6 is not null then 'NACIONAL'
else
	case 
		when SUBSTRING(A.PREFIJO,1,1)='4' then 'VISA'
		when SUBSTRING(A.PREFIJO,1,1)='2' then 'MC'
		when SUBSTRING(A.PREFIJO,1,1)='5' then 'MC'
		when SUBSTRING(A.PREFIJO,1,1)='6' then 'MC'
		when SUBSTRING(A.PREFIJO,1,1)='3' then 'AMEX'
		when SUBSTRING(A.PREFIJO,1,1)='7' then 'PRIV INT' 
		ELSE 'NACIONAL' END
	END AS MARCA,
sum(importe) as total,
sum(cantidad) as txns
into #temp_marca
from sac_ene.dbo.INFO_SAC as a
inner join sac.dbo.Adquirentes as aa on a.id_adquirente=aa.id_adquirente
left join SAC.dbo.bines_emisor as b on substring(a.PREFIJO,1,6)=b.bin6
GROUP BY aa.nombre_banco_grupo,
b.bin6,A.PREFIJO

--febrero

insert into #temp_marca
select 
aa.nombre_banco_grupo,
case 
when b.bin6 is not null then 'NACIONAL'
else
	case 
		when SUBSTRING(A.PREFIJO,1,1)='4' then 'VISA'
		when SUBSTRING(A.PREFIJO,1,1)='2' then 'MC'
		when SUBSTRING(A.PREFIJO,1,1)='5' then 'MC'
		when SUBSTRING(A.PREFIJO,1,1)='6' then 'MC'
		when SUBSTRING(A.PREFIJO,1,1)='3' then 'AMEX'
		when SUBSTRING(A.PREFIJO,1,1)='7' then 'PRIV INT' 
		ELSE 'NACIONAL' END
	END AS MARCA,
sum(importe) as total,
sum(cantidad) as txns
from sac_feb.dbo.INFO_SAC as a
inner join sac.dbo.Adquirentes as aa on a.id_adquirente=aa.id_adquirente
left join SAC.dbo.bines_emisor as b on substring(a.PREFIJO,1,6)=b.bin6
GROUP BY aa.nombre_banco_grupo,
b.bin6,A.PREFIJO

--por importe
select nombre_banco_grupo,MARCA,sum(total) as total from #temp_marca
group by nombre_banco_grupo,marca
order by nombre_banco_grupo,marca

--por transacciones
select nombre_banco_grupo,MARCA,sum(txns) as txns from #temp_marca
group by nombre_banco_grupo,marca
order by nombre_banco_grupo,marca

drop table #temp_marca

# Interredes

In [ ]:
use SAC_MAR
--Adquirente por dia por cve de transaccion
select 
a.fecha,
b.nombre_banco_grupo,
a.[cve_txn],
sum(a.[cantidad]) as txns,
sum(a.[IMPORTE]) as total
into #temp_totales
from INFO_SAC_INTERREDES as a
inner join sac.dbo.adquirentes as b on a.id_adquirente =b.id_adquirente
group by a.fecha,
b.nombre_banco_grupo,
a.[cve_txn]
select a.fecha,
a.nombre_banco_grupo,
(select isnull(sum(txns),0)  from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=101) as ventas_credito_txns,
(select isnull(sum(total),0) from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=101) as ventas_credito_total,
(select isnull(sum(txns),0)  from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=108) as devs_credito_txns,
(select isnull(sum(total),0) from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=108) as devs_credito_total,
(select isnull(sum(txns),0)  from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=103) as ventas_debito_txns,
(select isnull(sum(total),0) from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=103) as ventas_debito_total,
(select isnull(sum(txns),0)  from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=109) as devs_debito_txns,
(select isnull(sum(total),0) from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=109) as devs_debito_total,
(select isnull(sum(txns),0) from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=106) as cashback_txns,
(select isnull(sum(total),0) from #temp_totales where fecha= a.fecha and nombre_banco_grupo=a.nombre_banco_grupo and cve_txn=106) as cashback_total,
sum(txns) as txns,
sum(total) as total
from #temp_totales as a
group by a.fecha,a.nombre_banco_grupo
--totales por banco
select 
a.nombre_banco_grupo,
sum(total) as total,
sum(txns) as txns
from #temp_totales as a
group by a.nombre_banco_grupo
order by total desc
drop table #temp_totales